In [8]:
import openai
import pandas as pd
import numpy as np
import json
openai.api_key = "glhf_2c58a5c4be6a270fdd76f82e6179d2f1"
openai.api_base = "https://glhf.chat/api/openai/v1"
import time
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type





In [3]:
verbs = pd.read_csv('/Users/malhar.inamdar/Desktop/test/hindi-verbs.txt', header=None)
adj = pd.read_csv('/Users/malhar.inamdar/Desktop/test/adjectives_hindi.txt', header=None)
nouns = pd.read_csv('/Users/malhar.inamdar/Desktop/test/nouns_hindi.txt', header=None)
end = np.array(["happy", "sad", "mysterious"])

In [4]:
def generatePrompt():
    verb = verbs.sample(n=1).iloc[0, 0]
    noun = nouns.sample(n=1).iloc[0, 0]
    adjective = adj.sample(n=1).iloc[0, 0]

    ending = np.random.choice(end)
    # Randomly select one of verb, noun, or adjective to use in the prompt
    word_choice = np.random.choice(["verb", "noun", "adjective"])
    word = verb if word_choice == "verb" else noun if word_choice == "noun" else adjective

    basePromptHindi = f'''Write a simple Hindi children's story for a 5-year-old in 5-6 lines. The story should be inspired by Panchtantra tales and use easy, understandable language. Make it engaging with a clear beginning, middle, and end, and include only one brief dialogue between the characters. The story should have a gentle, natural resolution that reflects an emotion, such as happiness, friendship, or kindness, without explicitly using these words. Use the {word_choice} "{word}" naturally within the story.
    The story should naturally lead to a {ending} ending, conveyed through the characters’ reactions, atmosphere, or conclusion without directly using the word {ending}.
    Just give the marathi story'''
    newPrompt = f'''Write a simple Hindi children's story for a 5-year-old in 5-6 lines. The story should be inspired by traditional folk tales like Panchtantra, but can feature a mix of characters—such as children, family members, nature elements, or gentle imaginary beings—aside from animals. 
    Use easy and understandable language. Create an engaging flow with a clear beginning, middle, and end, and include one brief, friendly dialogue between characters. The story should end with a positive resolution, reflecting an emotion like joy, kindness, or curiosity through actions and reactions rather than direct words. 
    Naturally incorporate the {word_choice} "{word}" within the story. Ensure the story subtly leads to a {ending} ending through atmosphere and actions, without directly using the word {ending}. Just provide the Hindi story.'''
    return {'instruction':basePromptHindi, "word":word,"ending":ending}

In [ ]:
#  handling API calls with exponential backoff
@retry(
    wait=wait_exponential(multiplier=1, min=4, max=60),  # Start with 4s delay, double each time, max 60s
    stop=stop_after_attempt(5),  # Maximum 5 attempts
    retry=retry_if_exception_type((openai.error.RateLimitError, openai.error.APIError, openai.error.Timeout))
)
def make_api_call(prompt):
    return openai.ChatCompletion.create(
        model="hf:google/gemma-2-27b-it",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=500,
        top_p=0.9,
        frequency_penalty=0.5,
        presence_penalty=1
    )

In [ ]:
# method using exponential backoff
def store_dataset(n, batch_size=50):
    data = {
        "output": [],
        "input": [],
        "instruction": []
    }
    
    stories_generated = 0
    
    while stories_generated < n:
        batch_count = min(batch_size, n - stories_generated)
        print(f"Generating batch of {batch_count} stories. Progress: {stories_generated}/{n}")
        
        for _ in range(batch_count):
            try:
                prompt = generatePrompt()
                basePromptHindi = prompt['instruction']
                
                completion = make_api_call(basePromptHindi)
                response_message = completion.choices[0].message.content
                
                input_words = [prompt['word'], prompt['ending']]
                data["output"].append(response_message)
                data["input"].append(input_words)
                data["instruction"].append(basePromptHindi)
                stories_generated += 1
                
                # Save progress after each story
                with open('hindi23nov.json', 'w', encoding='utf-8') as json_file:
                    json.dump({"train": data}, json_file, ensure_ascii=False, indent=4)
                
            except Exception as e:
                print(f"Error generating story: {str(e)}")
                time.sleep(30)  # Additional cooldown on error
                continue
        
        # Add a small delay between batches to avoid hitting rate limits
        if stories_generated < n:
            time.sleep(5)
    
    print(f"Successfully generated {stories_generated} stories.")
    return data

In [ ]:
# Generate 1000 stories using exponential backoff
if __name__ == "__main__":
    store_dataset(1000)

In [ ]:
# conventionl method without exponential backoff
def store_dataset(n):
  data = {
    "output": [],
    "input": [],
    "instruction": []
  }
  for _ in range(n):
    prompt = generatePrompt()
    basePromptHindi = prompt['instruction']
    
    completion = openai.ChatCompletion.create(
      model="hf:google/gemma-2-27b-it",
      messages=[
        {"role": "user", "content": f"{basePromptHindi}"}
      ],
      temperature=0.3,
      max_tokens=500,
      top_p = 0.9,
      frequency_penalty=0.5,
      presence_penalty=1
    )
    response_message = completion.choices[0].message.content
    input_words = [prompt['word'],prompt['ending']]
    data["output"].append(response_message)
    data["input"].append(input_words)
    data["instruction"].append(basePromptHindi)
    with open('hindi23nov.json', 'w', encoding='utf-8') as json_file:
      json.dump({"train": data}, json_file, ensure_ascii=False, indent=4)
      

In [ ]:
store_dataset(1000)
